In [1]:
!pip3 install spacy >= 2.2.4

ERROR: Could not find a version that satisfies the requirement 2.2.4
ERROR: No matching distribution found for 2.2.4


In [2]:
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import praw
import streamlit as st


%matplotlib inline

In [3]:

reddit = praw.Reddit(client_id='qQBQxY9R1zGrYJH9pACBOw', client_secret='MjrjCout_Omkh_3feZFq3UrzyLBicw', user_agent='Crypto')


Version 7.4.0 of praw is outdated. Version 7.5.0 was released 6 days ago.


In [4]:
st.title('Crypto Trader Assistant')
st.text('This tool gives traders and investors a sentiment about the cryptocurrency from r/Cryptocurrency.')

ucurr = st.text_input('Cryptocurrency', 'BTC')
st.write('Cryptocurrency', ucurr)

2021-11-20 09:45:27.224 
  command:

    streamlit run /opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [5]:
crypto = []
ml_subreddit = reddit.subreddit('Cryptocurrency')
for post in ml_subreddit.hot(limit=30000):
    crypto.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
crypto = pd.DataFrame(crypto,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [6]:
crypto.dropna(subset=['body'], inplace=True)

2021-11-20 09:45:40.199 NumExpr defaulting to 8 threads.


In [7]:
crypto['original_body'] = crypto['body']

In [8]:
crypto

,title,score,id,subreddit,url,num_comments,body,created,original_body
0,"Daily Discussion - November 20, 2021 (GMT+0)",79,qxsxkw,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,4012,**Welcome to the Daily Discussion. Please read...,1.637366e+09,**Welcome to the Daily Discussion. Please read...
1,Decentraland is a dead metaverse and you're go...,1400,qxogsc,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,1044,I've followed MANA for a while now.\n\nOver th...,1.637353e+09,I've followed MANA for a while now.\n\nOver th...
2,FYI if you're searching meme hypecoins for the...,386,qxuhvh,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,253,I've been working in digital marketing for alm...,1.637371e+09,I've been working in digital marketing for alm...
3,Predictions Tournament,9992,qxp9re,CryptoCurrency,https://reddit.com/r/CryptoCurrency/prediction...,225,,1.637316e+09,
4,I absolutely do not sympathize with people who...,3555,qxepgd,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,1107,My friend lost $2K in the whole squid game tok...,1.637323e+09,My friend lost $2K in the whole squid game tok...
...,...,...,...,...,...,...,...,...,...
636,"Crime at crypto ""DeFi"" sites hits US$10.5 bill...",2,qxfmw3,CryptoCurrency,https://www.channelnewsasia.com/business/crime...,6,,1.637327e+09,
637,TIME Magazine to hold ETH on Balance Sheet.,33,qx0qwb,CryptoCurrency,https://cryptobriefing.com/time-magazine-to-ho...,20,,1.637273e+09,
638,Yesterday I loaded up my Hardware Wallet with ...,12,qx69si,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,59,What tactics are you using to keep yourself fr...,1.637290e+09,What tactics are you using to keep yourself fr...
639,What's the stupidest coin you know?,15,qx5ck4,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,53,Today I was checking new coins on CoinMarketCa...,1.637287e+09,Today I was checking new coins on CoinMarketCa...


In [9]:
import spacy
nlp = spacy.blank('en')


import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text):
    doc = nlp(text)
    nlp.Defaults.stop_words -= {"nt","crypto", "cryptocurrency"}
    return " ".join([token.text for token in doc if not token.is_stop])

def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

remove_spaces = lambda x : re.sub('\\s+', ' ', x)

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_double_quotes = lambda x : x.replace('"', '')
remove_single_quotes = lambda x : x.replace('\'', '')
trim = lambda x : x.strip()

other_chars = ['*', '#', '&x200B', '[', ']', '; ',' ;' "&nbsp", "“","“","”", "x200b"]


def remove_other_chars(x: str):
    for char in other_chars:
        x = x.replace(char, '')
    
    return x


def lower_case_text(text):
    return text.lower()

funcs = [
    remove_urls, 
    remove_punctuation,
    remove_stop_words, 
    remove_emoji, 
    remove_double_quotes, 
    remove_single_quotes,
    lower_case_text,
    remove_other_chars,
    # lemmatize_words,
    remove_spaces,
    trim]



In [10]:
for fun in funcs:
    crypto['body'] = crypto['body'].apply(fun)

In [11]:
crypto.reset_index(inplace=True)
crypto.drop(['index'], axis=1, inplace=True)



In [12]:
body_list = crypto.body.tolist()

In [13]:
def generate_ngrams(text, n_gram=2):
    token = [token for token in text.lower().split(' ') if token != '']
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

In [14]:
!pip3 install fbpca

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import linalg
from sklearn import decomposition
import fbpca

number_of_topics = 30
num_top_words = 20
vectorizer = TfidfVectorizer()

In [16]:
bigram_vectorizer = TfidfVectorizer(ngram_range=(2,2))

In [17]:
bigrams_vectors = bigram_vectorizer.fit_transform(body_list).todense()
bigrams_vocab = np.array(bigram_vectorizer.get_feature_names())

# LDA

In [45]:
docs = [generate_ngrams(body) for body in body_list]

In [46]:
docs

[['welcome daily',
  'daily discussion',
  'discussion read',
  'read disclaimer',
  'disclaimer guidelines',
  'guidelines rules',
  'rules participating',
  'participating disclaimer',
  'disclaimer consider',
  'consider information',
  'information posted',
  'posted liberal',
  'liberal heaps',
  'heaps salt',
  'salt cross',
  'cross check',
  'check information',
  'information read',
  'read thread',
  'thread known',
  'known sources',
  'sources trade',
  'trade information',
  'information posted',
  'posted open',
  'open thread',
  'thread highly',
  'highly misleading',
  'misleading attempt',
  'attempt manipulate',
  'manipulate new',
  'new readers',
  'readers known',
  'known pump',
  'pump dump',
  'dump pnd',
  'pnd groups',
  'groups profit',
  'profit beware',
  'beware practices',
  'practices exercise',
  'exercise utmost',
  'utmost caution',
  'caution acting',
  'acting trade',
  'trade tip',
  'tip mentioned',
  'mentioned careful',
  'careful information',

In [47]:
from gensim.corpora import Dictionary

dic = Dictionary(docs)

2021-11-20 10:32:16.355 adding document #0 to Dictionary(0 unique tokens: [])
2021-11-20 10:32:16.422 built Dictionary(36568 unique tokens: ['2fa possible', 'acting trade', 'actions share', 'afford lose', 'age requirements']...) from 641 documents (total 41874 corpus positions)
2021-11-20 10:32:16.423 Dictionary lifecycle event {'msg': "built Dictionary(36568 unique tokens: ['2fa possible', 'acting trade', 'actions share', 'afford lose', 'age requirements']...) from 641 documents (total 41874 corpus positions)", 'datetime': '2021-11-20T10:32:16.423424', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [105]:
id = 0

for i in dic:
    if dic[i] == r'^crypto|$crypto':
        id = i

In [106]:
id

0

In [82]:
dic[1]

'acting trade'

In [48]:
dic.dfs

{130: 2,
 21: 3,
 26: 2,
 92: 2,
 24: 2,
 38: 2,
 100: 2,
 70: 2,
 23: 2,
 18: 2,
 42: 2,
 76: 2,
 53: 2,
 39: 2,
 102: 2,
 19: 2,
 14: 2,
 43: 2,
 93: 3,
 118: 2,
 51: 2,
 114: 2,
 122: 2,
 77: 2,
 68: 2,
 117: 2,
 40: 2,
 60: 2,
 7: 2,
 57: 2,
 64: 2,
 94: 2,
 50: 2,
 84: 2,
 31: 2,
 72: 2,
 37: 2,
 82: 2,
 10: 2,
 78: 2,
 34: 2,
 128: 2,
 13: 2,
 1: 2,
 123: 2,
 120: 2,
 58: 2,
 12: 2,
 44: 2,
 108: 2,
 2: 2,
 109: 2,
 5: 2,
 74: 2,
 110: 2,
 71: 2,
 111: 2,
 81: 4,
 49: 2,
 129: 2,
 107: 2,
 126: 2,
 115: 2,
 66: 2,
 0: 2,
 75: 2,
 11: 2,
 103: 2,
 112: 2,
 46: 5,
 3: 9,
 56: 2,
 35: 2,
 85: 2,
 106: 2,
 83: 2,
 124: 2,
 98: 2,
 36: 2,
 105: 2,
 17: 2,
 104: 2,
 101: 2,
 116: 2,
 99: 2,
 6: 2,
 119: 2,
 80: 2,
 33: 2,
 48: 2,
 4: 2,
 96: 2,
 55: 2,
 32: 2,
 27: 2,
 121: 2,
 95: 3,
 20: 2,
 9: 2,
 15: 2,
 73: 2,
 125: 2,
 67: 2,
 86: 2,
 41: 2,
 52: 2,
 16: 2,
 113: 2,
 65: 2,
 127: 2,
 54: 2,
 8: 2,
 97: 2,
 45: 2,
 89: 2,
 62: 3,
 63: 2,
 131: 2,
 69: 2,
 87: 2,
 25: 2,
 90: 2,
 7

In [49]:
corpus = [dic.doc2bow(doc) for doc in docs]

In [50]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1)

## Training 

Now it's time to train our topic model. We do this with the following parameters:

- **corpus**: the bag-of-word representations of our documents
- **id2token**: the mapping from indices to words
- **num_topics**: the number of topics we want the model to identify
- **chunksize**: the number of documents the model sees for every update
- **passes**: the number of times we show the total corpus to the model during training
- **random_state**: we use a seed to ensure reproducibility.

In [107]:
from gensim.models import LdaModel

model = LdaModel(corpus=corpus, id2word=dic, num_topics=number_of_topics, chunksize=2500, passes=5, random_state=1)

2021-11-20 11:33:29.795 using symmetric alpha at 0.03333333333333333
2021-11-20 11:33:29.799 using symmetric eta at 0.03333333333333333
2021-11-20 11:33:29.810 using serial LDA version on this node
2021-11-20 11:33:29.900 running online (multi-pass) LDA training, 30 topics, 5 passes over the supplied corpus of 641 documents, updating model once every 641 documents, evaluating perplexity every 641 documents, iterating 50x with a convergence threshold of 0.001000
2021-11-20 11:33:29.901 too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-20 11:33:30.284 -63.349 per-word bound, 11748315625575661568.0 perplexity estimate based on a held-out corpus of 641 documents with 41874 words
2021-11-20 11:33:30.285 PROGRESS: pass 0, at document #641/641
2021-11-20 11:33:30.547 topic #20 (0.033): 0.006*"24 hours" + 0.001*"total monetary" + 0.001*"portfolio value" + 0.001*"limit orders" + 0.001*"monetary value" + 0.001*"sp 500

In [108]:
model.get_term_topics(130, minimum_probability=None)

[]

In [95]:
# x=model.show_topics(num_topics=12, num_words=5,formatted=False)
# topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# tid = null

# for topic,words in topics_words:
#     print(str(topic)+ ":"+ str(words))
#     for word in words
#         if word == 'btc'
#             tid = topic
#             break

# prin

5::['avalanche', 'price', 'blockchain', 'credit', 'week']
7::['crypto', '…', 'blockchain', 'card', 'like']
15::['nt', 'crypto', 'time', 'market', 'btc']
10::['market', 'nt', 'like', 'crypto', 'new']
22::['value', 'portfolio', 'total', '€', 'profits']
25::['crypto', 'nt', 'buy', 'average', 'inu']
6::['…', 'like', 'people', 'metaverse', 'think']
19::['smart', 'dao', 'contracts', 'nt', 'contract']
2::['nt', 'account', 'payment', 've', 'told']
29::['crypto', 'nt', 'like', 'coins', 'new']
24::['·', 'market', 'btc', 'bitcoin', 'know']
9::['crypto', 'people', 'nt', 'money', 'earn']

avalanche price blockchain credit week
crypto … blockchain card like
nt crypto time market btc
market nt like crypto new
value portfolio total € profits
crypto nt buy average inu
… like people metaverse think
smart dao contracts nt contract
nt account payment ve told
crypto nt like coins new
· market btc bitcoin know
crypto people nt money earn


In [94]:
x=model.show_topics()

twords={}
for topic,word in x:
    twords[topic]= re.split('[^A-Za-z ]+', '', word)
print(twords)

TypeError: 'str' object cannot be interpreted as an integer

In [109]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words, '\n\n')

2021-11-20 11:33:44.189 topic #5 (0.033): 0.001*"buy dip" + 0.001*"block chain" + 0.001*"5 services" + 0.001*"long term" + 0.001*"ve seen" + 0.001*"3 months" + 0.001*"absolutely reason" + 0.001*"reason panic" + 0.001*"11 days" + 0.001*"wedge pattern"
2021-11-20 11:33:44.191 topic #13 (0.033): 0.002*"new investors" + 0.001*"immutable x" + 0.001*"gas fees" + 0.001*"long run" + 0.001*"financial advice" + 0.001*"lose money" + 0.001*"market manipulation" + 0.001*"m talking" + 0.001*"gon na" + 0.001*"energy time"
2021-11-20 11:33:44.192 topic #1 (0.033): 0.003*"smart contract" + 0.001*"buy dip" + 0.001*"s reason" + 0.001*"middle month" + 0.001*"dip middle" + 0.001*"people complaining" + 0.001*"good bad" + 0.001*"nt want" + 0.001*"nft games" + 0.001*"money left"
2021-11-20 11:33:44.194 topic #2 (0.033): 0.001*"meme coins" + 0.001*"technical analysis" + 0.001*"feel free" + 0.001*"long term" + 0.001*"stock market" + 0.001*"nt want" + 0.001*"experience stock" + 0.001*"disable follows" + 0.001*"c

6 : 0.001*"buy dip" + 0.001*"block chain" + 0.001*"5 services" + 0.001*"long term" + 0.001*"ve seen" + 0.001*"3 months" + 0.001*"absolutely reason" + 0.001*"reason panic" + 0.001*"11 days" + 0.001*"wedge pattern" 


14 : 0.002*"new investors" + 0.001*"immutable x" + 0.001*"gas fees" + 0.001*"long run" + 0.001*"financial advice" + 0.001*"lose money" + 0.001*"market manipulation" + 0.001*"m talking" + 0.001*"gon na" + 0.001*"energy time" 


2 : 0.003*"smart contract" + 0.001*"buy dip" + 0.001*"s reason" + 0.001*"middle month" + 0.001*"dip middle" + 0.001*"people complaining" + 0.001*"good bad" + 0.001*"nt want" + 0.001*"nft games" + 0.001*"money left" 


3 : 0.001*"meme coins" + 0.001*"technical analysis" + 0.001*"feel free" + 0.001*"long term" + 0.001*"stock market" + 0.001*"nt want" + 0.001*"experience stock" + 0.001*"disable follows" + 0.001*"chats messages" + 0.001*"according twitter" 


24 : 0.001*"bear run" + 0.001*"btc eth" + 0.001*"trying create" + 0.001*"new bedrock" + 0.001*"do

In [24]:
original_body_list = crypto.original_body.tolist()

In [25]:
for (text, doc) in zip(original_body_list[:9], docs[:9]):
    print('\033[1m' + 'Text: ' + '\033[0m', text)
    print('\033[1m' + 'Topics: ' + '\033[0m', [(topic+1, prob) for (topic, prob) in model[dic.doc2bow(doc)] if prob > 0.15])
    print('\n')

Text:  **Welcome to the Daily Discussion. Please read the disclaimer, guidelines, and rules before participating.**
 
***
-
    
###Disclaimer:

Consider all information posted here with several liberal heaps of salt, and always cross check any information you may read on this thread with known sources. Any trade information posted in this open thread may be highly misleading, and could be an attempt to manipulate new readers by known "pump and dump (PnD) groups" for their own profit. BEWARE of such practices and exercise utmost caution before acting on any trade tip mentioned here. 


**Please be careful about what information you share and the actions you take.** Do not share the amounts of your portfolios (why not just share percentage?). Do not share your private keys or wallet seed. Use strong, non-SMS 2FA if possible. Beware of scammers and be smart. Do not invest more than you can afford to lose, and do not fall for pyramid schemes, promises of unrealistic returns (get-rich-quic

Now let's see what topic is going to assign to the post-**2179**, previously tested with the svd decomposition for the unigram term-document matrix.

In [27]:
print('\033[1m' + 'Text: ' + '\033[0m', original_body_list[20])
print('\033[1m' + 'Topic: ' + '\033[0m', [(topic+1, prob) for (topic, prob) in model[dic.doc2bow(docs[20])] if prob > 0.1])

Text:  It noticed something strange the last few days. The dip of the market just did nothing to me.

I have a fair share of my savings in crypto, im here to have something for my retirement down the line and some gains on the way. 

I know prices will go up in the long run. Im still interested in news and how everything develops and unfolds.

But there are no negativ feeling, no panic or urge to sell. Its like crypto is doing its thing again, it will calm down eventually.
It was the same in the late spring dip.

I just dont care right now. Ill keep dca-ing, ill keep looking for the next thing, but i have so much faith into crypto, dips dont scare me at all.

Am i naive? Do I need to experience fud and a crypto-winter?

I dont know. Time will tell. Until then ill lay back and chill.
Topic:  [(4, 0.98440534)]


the topics assigned by the LDA are not 100% spot-on, **topic 5** seems to be appropriate but **topic 9** seems to be out of place.

Let's see what kind of topic the LDA is going to choose for the post **280** tested with the bigram matrix decomposition.

In [ ]:
print('\033[1m' + 'Text: ' + '\033[0m', original_body_list[30])
print('\033[1m' + 'Topic: ' + '\033[0m',[(topic+1, prob) for (topic, prob) in model[dic.doc2bow(docs[30])] if prob > 0.1])

Text:  My DCA day (pay day) is still half a month away and greed literally burns me from the inside because I think I can miss this dip.

I always try to catch the moment and buy at the lowest possible price, and it's a damn shame to watch such a great opportunity to load my bags (before price skyrocket) pass me by.

How do you guys deal with the feeling of lost profits at such moments? I understand that I have no choice but to just relax and continue watching the charts, but it's kinda difficult

p.s. looks like my love for crypto is becoming an addiction lol
Topic:  [(4, 0.9817565)]


In [28]:
from textblob import TextBlob
# create a function to get subjectivity
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity

# create a function to get the polarity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity

# create two new columns called "Subjectivity" & "Polarity"
crypto['subjectivity'] = crypto['original_body'].apply(getSubjectivity)
crypto['polarity'] = crypto['original_body'].apply(getPolarity)

ModuleNotFoundError: No module named 'textblob'

In [ ]:
# create a function get the sentiment text
def getSentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [ ]:
# create a column to store the text sentiment
crypto['sentiment'] = crypto['polarity'].apply(getSentiment)



,title,score,id,subreddit,url,num_comments,body,created,original_body,subjectivity,polarity,sentiment
0,"Daily Discussion - November 19, 2021 (GMT+0)",186,qx37qc,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,8124,welcome daily discussion read disclaimer guide...,1.637280e+09,**Welcome to the Daily Discussion. Please read...,0.430163,0.094759,positive
1,What's the next metaverse play with high ROI l...,871,qx73x6,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,567,okay ton research … 3k spend tomorrow kind wan...,1.637292e+09,Okay just did a ton of research… I have $3k to...,0.476463,0.129577,positive
2,ADOPTION: The richest person in crypto chooses...,281,qxbosh,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,171,sam bankmanfried ceo alameda research chosen c...,1.637310e+09,Sam Bankman-Fried who is the CEO of Alameda Re...,0.525571,0.178671,positive
3,I absolutely do not sympathize with people who...,140,qxepgd,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,133,friend lost 2 k squid game token fiasco later ...,1.637323e+09,My friend lost $2K in the whole squid game tok...,0.480895,0.090294,positive
4,Now is your chance to actually buy the dip,3174,qwxint,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,1148,says late beg dip chance asked dollar cost ave...,1.637264e+09,Everyone says they’re “late” and beg for a dip...,0.413333,0.153333,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
655,Is current state of crypto currency a prelude ...,7,qwp793,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,17,crypto saga began saw effects unbridled greed ...,1.637240e+09,The crypto saga began after we saw the effects...,0.414867,0.123556,positive
656,IOTEX 2.0: The Rise of MachineFi,12,qwmxnz,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,13,iotex actually night announced machinefi follo...,1.637231e+09,"IOTEX have just (actually, last night) announc...",0.610309,0.130712,positive
657,This is why $AVAX is killing the Market right ...,4,qwrbry,CryptoCurrency,/r/Avax/comments/qw5ce1/this_is_why_avax_is_ki...,1,,1.637247e+09,,0.000000,0.000000,neutral
658,How can I/we help you? Vol. 2,2,qwxvgp,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,21,previous post similar nature got attention lot...,1.637265e+09,My [previous](https://old.reddit.com/r/CryptoC...,0.480208,-0.005208,negative


In [ ]:
if __name__ == ‘__main__‘:
    main()
    import warnings
    warnings.filterwarnings(‘ignore’)